In [1]:
import numpy as np
import torch
import pandas as pd
import glob
import yaml
import os
import math
import scipy.io as sio
import shutil
from pathlib import Path

from plotting import *
from utils import *
from TMP_model import MP_model,TestTMPModel

In [2]:
# Extract motion of videos for each subject and save all subfiles
#  in a folder same name of videos in the same directory of original video 

In [4]:
# List of all subjects to process
all_subjects = [1,2,3,4,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
            31,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]

completed_subjects = [1,2,10,11,12,13,15,16,17,18,19,22, 20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36]
subjects = [4,5,8,9,50,51,52,53 ,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,75,76,77,80,
            81,82,83,84,85,87,88,89,90]

v3d_base_path = '../../data/F_Subjects/'
video_base_path = '../../data/newData/'

for subject_id in subjects:
    v3d_file = f'{v3d_base_path}F_v3d_Subject_{subject_id}.mat'
    video_file = f'{video_base_path}F_PG1_Subject_{subject_id}_L.avi'
    
    # Process the subject
    try:
        main_rub = sio.loadmat(v3d_file, squeeze_me=True, struct_as_record=False)
        single_videos(video_file, main_rub)
        print(f"Successfully processed Subject {subject_id}")
    except Exception as e:
        print(f"Error processing Subject {subject_id}: {e}")

Created output directory: ../../data/newData\F_PG1_Subject_4_L
Available motions in this file:
Saved motion mapping with 43 entries.
Video processing complete. All segments saved to ../../data/newData\F_PG1_Subject_4_L
Successfully processed Subject 4
Created output directory: ../../data/newData\F_PG1_Subject_5_L
Available motions in this file:
Saved motion mapping with 43 entries.
Video processing complete. All segments saved to ../../data/newData\F_PG1_Subject_5_L
Successfully processed Subject 5
Created output directory: ../../data/newData\F_PG1_Subject_8_L
Available motions in this file:
Saved motion mapping with 43 entries.
Video processing complete. All segments saved to ../../data/newData\F_PG1_Subject_8_L
Successfully processed Subject 8
Created output directory: ../../data/newData\F_PG1_Subject_9_L
Available motions in this file:
Saved motion mapping with 43 entries.
Video processing complete. All segments saved to ../../data/newData\F_PG1_Subject_9_L
Successfully processed Su

Now, in this step, I give these extracted motion videos to MMpose on the server
to get 3d pose json files on server:

#### Data Processing Workflow on server

1. Connect to VPN

2. Connect to remote server via SSH with port forwarding:
   ```bash
   ssh -p 38954 -L 8888:localhost:8888 arefeh@130.15.106.49
   ```
3. Activate virtual environment:
   ```bash
   source ~/myenv/bin/activate
   ```
4. Start Jupyter notebook on remote server:
   ```bash
   jupyter notebook --no-browser
   ```
5. In a new terminal session, transfer all video folders (all at once)):
   ```bash
   scp -r -P 38954 F_PG1_Subject_*_L arefeh@130.15.106.49:/data1/users/arefeh/movementProject/MMpose/videos/
   ```
6. Edit the main.py file on server to update subject IDs:
   ```bash
   # Path: movementProject/MMpose/main.py
   # Update the subject IDs as needed
   ```
7. Open a screen session:
   ```bash
   screen -r mysession
   ```
8. Run the main script:
   ```bash
   python main.py
   ```
9. Detach from screen session (to let it run in background):
   ```bash
   # Press Ctrl+A and then D
   ```
10. After processing completes, transfer results back to local machine:
    ```bash
    scp -r -P 38954 arefeh@130.15.106.49:/data1/users/arefeh/movementProject/MMpose/pred_out ./data/MMpose/pred_out
    ```


Now I load separated motion vidoes json file from MMpose on server,
merge them in fixed order based on common motion_mapping
and concatenate them in one dataframe for each subject

In [2]:
import json
subjects = [5,8,9,10,11,12,13,15,16,17,18,19,22, 20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,4,50,51,52,53 ,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,75,76,77,81,82,83,87,88,89,90]
            # 4,5,8,9,10,11,12,13,15,16,17,18,19,22, 20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53 ,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,75,76,77,80,81,82,83,84,85,87,88,89,90
source_folder = '../../data/MMpose/pred_out'
destination_folder = "../../data/MMpose/df_files_3d"
visualization_folder = "../../data/MMpose/visualizations"

# Load motion mapping
with open('../../data/motion_mapping.json', 'r') as f:
    motion_mapping = json.load(f)['mapping']
    # Invert mapping for easier lookup
    id_to_motion = {str(v): k for k, v in motion_mapping.items()}

# Define equivalent motions - map to canonical ID
equivalent_motions = {
    # Format: ID: canonical_ID
    "11": "11",  # cross_arms -> cross_arms
    "24": "11",  # crossarms -> cross_arms (maps motion 24 to 11)
    "12": "12",  # jumping_jacks -> jumping_jacks
    "23": "12",  # jumping_jack -> jumping_jacks (maps motion 23 to 12)
}

subject_motions, common_motions = create_csv_from_json(
    source_folder, destination_folder, subjects, equivalent_motions, id_to_motion
)
    
## merge sequence of common motions for each subject to one csv dataframe
# merge_subject_csv_files(destination_folder, merged_folder, subjects, common_motions)

## create_common_motions_json file for future record
common_mapping = {}
output_json_path = '../../data/common_motion_mapping.json'
for motion_id in common_motions:
    motion_id_str = str(motion_id)
    if motion_id_str in id_to_motion:
        motion_name = id_to_motion[motion_id_str]
        common_mapping[motion_name] = motion_id

common_motion_data = {"mapping": common_mapping}
with open(output_json_path, 'w') as f:
    json.dump(common_motion_data, f, indent=4)


Common motions across all subjects: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20]
Motion names: ['hand_clapping', 'jogging', 'walking', 'cross_legged_sitting', 'vertical_jumping', 'crawling', 'hand_waving', 'running_in_spot', 'checking_watch', 'kicking', 'taking_photo', 'cross_arms', 'jumping_jacks', 'scratching_head', 'throw/catch', 'pointing', 'sitting_down', 'sideways', 'phone_talking', 'stretching']


In [4]:
with open('../../data/common_motion_mapping.json', 'r') as f:
    common_motion_mapping = json.load(f)['mapping']
    id_to_motion = {str(v): k for k, v in common_motion_mapping.items()}

# for k, v in common_motion_mapping.items():
#     print(k)

##### Now we have 3d pose data csv files in same order motion videos for all subjects
##### Visualize some frames in one  csv file to verify 3d data

##### Now we need to convert csv files (which contain all motions together) to bvh format (joint angle trajectories) so we can use for TMP model
##### For this part, we will use the method in video2bvh toolbox (https://github.com/KevinLTT/video2bvh)

In [12]:
from h36m_csv_converter import H36MConverter

converter = H36MConverter()
path = Path(r"/Users/arefehfarahmandi/PycharmProjects/Movement_Recognition/data/MMpose/df_files_3d")

# Create output directory for BVH files
output_dir = Path(r"/Users/arefehfarahmandi/PycharmProjects/Movement_Recognition/data/bvh_files")
output_dir.mkdir(exist_ok=True)

with open('../../data/common_motion_mapping.json', 'r') as f:
    common_motion_mapping = json.load(f)['mapping']

for csv_file in path.glob("*.csv"):
    bvh_file = output_dir / csv_file.name.replace(".csv", ".bvh")

    print(f"Converting {csv_file.name} to BVH...")
    try:
        channels, header = converter.convert_to_bvh(str(csv_file), str(bvh_file))
    except Exception as e:
        print(f"Error converting {csv_file.name}: {str(e)}")

print("\nConversion complete!")

Converting subject_70_motion_08.csv to BVH...
Converting subject_59_motion_18.csv to BVH...
Converting subject_64_motion_19.csv to BVH...
Converting subject_70_motion_20.csv to BVH...
Converting subject_16_motion_11.csv to BVH...
Converting subject_71_motion_12.csv to BVH...
Converting subject_58_motion_02.csv to BVH...
Converting subject_71_motion_06.csv to BVH...
Converting subject_58_motion_16.csv to BVH...
Converting subject_16_motion_05.csv to BVH...
Converting subject_82_motion_01.csv to BVH...
Converting subject_5_motion_01.csv to BVH...
Converting subject_65_motion_17.csv to BVH...
Converting subject_65_motion_03.csv to BVH...
Converting subject_15_motion_14.csv to BVH...
Converting subject_15_motion_00.csv to BVH...
Converting subject_81_motion_04.csv to BVH...
Converting subject_66_motion_12.csv to BVH...
Converting subject_28_motion_01.csv to BVH...
Converting subject_66_motion_06.csv to BVH...
Converting subject_81_motion_10.csv to BVH...
Converting subject_67_motion_20.csv

##### Visualize joint angle trajectories with and without smoothing

In [14]:
from visualize_joint_angles import *
bvh_file = "../../data/bvh_files/subject_1_motion_02.bvh"

# Customize these joints based on what you want to analyze
joints_to_analyze = ['LeftWrist', 'RightWrist', 'LeftKnee', 'RightKnee', 'LeftAnkle']

# First, visualize original data
print("\nStep 1: Analyzing original data...")
result_original = visualize_joint_angles_with_smoothing(
    bvh_file,
    joints_to_analyze,
    apply_smoothing=False
)

# Then, apply smoothing and visualize
print("\n Step 2: Applying Butterworth smoothing and comparing...")
result_smoothed = visualize_joint_angles_with_smoothing(
    bvh_file,
    joints_to_analyze,
    apply_smoothing=True,
    cutoff_freq=6.0,  # 6 Hz as recommended by research
    filter_order=6  # Sixth-order as recommended
)



Step 1: Analyzing original data...

 Plotting results:

📊 Plot saved as: figures/subject_1_motion_02_joint_angles_original.png

 Step 2: Applying Butterworth smoothing and comparing...

Applying smoothing...

 Plotting results:
   LeftWrist.Zrotation: RMSE=0.000°, Correlation=0.695, Noise reduction=33.8%
   LeftWrist.Xrotation: RMSE=0.000°, Correlation=0.667, Noise reduction=36.8%
   LeftWrist.Yrotation: RMSE=0.000°, Correlation=0.678, Noise reduction=35.4%
   RightWrist.Zrotation: RMSE=0.000°, Correlation=0.559, Noise reduction=48.1%
   RightWrist.Xrotation: RMSE=0.000°, Correlation=0.630, Noise reduction=40.2%
   RightWrist.Yrotation: RMSE=0.000°, Correlation=0.676, Noise reduction=34.7%
   LeftKnee.Zrotation: RMSE=0.314°, Correlation=0.995, Noise reduction=0.6%
   LeftKnee.Xrotation: RMSE=0.886°, Correlation=0.998, Noise reduction=0.2%
   LeftKnee.Yrotation: RMSE=0.636°, Correlation=0.995, Noise reduction=0.6%
   RightKnee.Zrotation: RMSE=0.268°, Correlation=0.996, Noise reduction=

##### Apply  low-pass filter with a 6th-order Butterworth filter (6-Hz cut-off frequency) on trajectories
##### Segmentation step:  joint angles of each movement sequence partitioned into segments in the temporal dimension.



In [ ]:
from plotting import *
folder_path = "../../data/bvh_files"
bvh_data,_ = read_bvh_files(folder_path)

# Process data according (smoothing and segmentation)
processed_data= process_bvh_data(bvh_data)

In [ ]:


#segment videos of each movements across all subjects
# exceptions = ["crawling","walking","hand_waving","vertical_jumping"]
for motion_name, motion_id_str in common_motion_mapping.items():
    motion_id = int(motion_id_str)
    print(f"&&&&&  {motion_name}   &&&&&")
    for i in subjects:
        segmentation(motion_id,subject_id=i)

&&&&&  hand_clapping   &&&&&
Segment frames: [(0, 17), (17, 46), (46, 65), (65, 85)]
Segment frames: [(0, 18), (18, 46), (46, 66)]
Segment frames: [(0, 7), (7, 37), (37, 61), (61, 82), (82, 119), (119, 122)]
Segment frames: [(0, 4), (4, 30), (30, 63), (63, 83), (83, 108)]
Segment frames: [(0, 2), (2, 31), (31, 57), (57, 82), (82, 108), (108, 109)]
Segment frames: [(0, 20), (20, 59), (59, 84), (84, 109), (109, 143), (143, 183), (183, 184)]
Segment frames: [(0, 3), (3, 25), (25, 54), (54, 88), (88, 115), (115, 117)]
Segment frames: [(0, 2), (2, 21), (21, 41), (41, 78), (78, 80)]
Segment frames: [(0, 2), (2, 22), (22, 41), (41, 68), (68, 69)]
Segment frames: [(0, 17), (17, 43), (43, 66), (66, 90), (90, 92)]
Segment frames: [(0, 3), (3, 31), (31, 53), (53, 73), (73, 102), (102, 105)]
Segment frames: [(0, 2), (2, 33), (33, 58), (58, 81), (81, 117), (117, 136), (136, 147)]
Segment frames: [(0, 2), (2, 34), (34, 53), (53, 72), (72, 92)]
Segment frames: [(0, 11), (11, 31), (31, 50), (50, 79), 

In [48]:
# Add this after finding the segment boundaries and before creating segments
# Create a plot showing the first 5 joint trajectories with segment boundaries

plt.figure(figsize=(15, 10))

# Select 5 joints to display
display_joints = ['LWrist', 'RWrist', 'LAnkle', 'RAnkle', 'Hip']
coords = ['x_3d', 'y_3d', 'z_3d']
colors = ['r', 'g', 'b']

# Create subplots for each joint
for j_idx, joint in enumerate(display_joints):
    plt.subplot(len(display_joints), 1, j_idx+1)

    # Get data for this joint
    joint_data = df_3d[df_3d['joint_name'] == joint]

    # Sort by frame
    joint_data = joint_data.sort_values('frame_id')

    # Plot each coordinate (x, y, z)
    for c_idx, coord in enumerate(coords):
        plt.plot(joint_data['frame_id'], joint_data[coord],
                 color=colors[c_idx], label=f'{coord}', alpha=0.7)

    # Add segment boundary lines
    boundary_frames = [sorted_frames[idx] for idx in segment_boundaries]
    for b_frame in boundary_frames:
        plt.axvline(x=b_frame, color='k', linestyle='--', alpha=0.5)

    # Set title and labels
    plt.title(f'{joint} Trajectory')
    plt.ylabel('Position')

    # Only add legend to the first subplot to avoid redundancy
    if j_idx == 0:
        plt.legend()

# Set xlabel only for the bottom subplot
plt.xlabel('Frame')

plt.tight_layout()
plt.savefig(os.path.join(destination_folder, 'joint_trajectories_with_boundaries.png'))
plt.close()